# Enhance MagIC file for Shaat dike data

Two MagIC files are generated from project research:
- The MagIC file exported from Demag_GUI associated with the thermal demagnetization data
- The MagIC file exported from the IRM database associated with the rock magnetic experiments

These files need to be merged together into a single contribution as was done in:
https://github.com/Swanson-Hysell-Group/2021_ECMB/blob/main/data/rockmag_pmag_MagIC/combine_MagIC.ipynb

## Import Python packages

In [1]:
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
pmag_dir_path = '../data/pmag/demag'
pmag_contribution = cb.Contribution(pmag_dir_path)
pmag_measurements = pmag_contribution.tables['measurements'].df
pmag_specimens = pmag_contribution.tables['specimens'].df
pmag_samples = pmag_contribution.tables['samples'].df
pmag_sites = pmag_contribution.tables['sites'].df
pmag_locations = pmag_contribution.tables['locations'].df

-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


## Enhance the site location precision and add site metadata

An issue with the CIT data format is that the precision on the site longitude/latitude is low given the ASCII file format. We therefore need to bring in more precise locations which are within the site_locations.csv file and add them to the site table.

Additionally, MagIC requires that sites have additional metadata including:
- geologic_classes
- geologic_types
- lithologies
- age (or age_high and age_low)
- age_unit
  
These data are also in the site_locations.csv file and can be merged into the sites table.

In [3]:
# get more precise site locations
SD_site_locations = pd.read_csv('../data/field_data/site_locations.csv', sep=',', header=0, index_col='site')
# map site locations to site data
pmag_sites['lat'] = SD_site_locations['latitude'][pmag_sites['site']].tolist()
pmag_sites['lon'] = SD_site_locations['longitude'][pmag_sites['site']].tolist()
pmag_sites['geologic_classes'] = SD_site_locations['geologic_classes'][pmag_sites['site']].tolist()
pmag_sites['geologic_types'] = SD_site_locations['geologic_types'][pmag_sites['site']].tolist()
pmag_sites['lithologies'] = SD_site_locations['lithologies'][pmag_sites['site']].tolist()
pmag_sites['age_high'] = SD_site_locations['age_high'][pmag_sites['site']].tolist()
pmag_sites['age_low'] = SD_site_locations['age_low'][pmag_sites['site']].tolist()
pmag_sites['age_unit'] = SD_site_locations['age_unit'][pmag_sites['site']].tolist()

### Filter out tilt corrected sites when not in Leger panel

We only have structural control in the Leger panel. We exported both geographic and tilt-corrected coordinates for every site out of Pmag_GUI, but should only report those for the Leger panel. We can find the sites within the Leger panel and only keep pmag_sites['dir_tilt_correction'] of 100 for those sites.

In [4]:
leger_sites = SD_site_locations[SD_site_locations['locale'].str.contains("Leger", na=False)].index
leger_sites

Index(['SD36', 'SD37', 'SD38', 'SD39', 'SD40', 'SD42', 'SD43', 'SD44', 'SD45',
       'SD46', 'SD47', 'SD48', 'SD49', 'SD50', 'SD51', 'SD52', 'SD53', 'SD54',
       'SD55', 'SD56', 'SD57', 'SD58', 'SD59', 'SD60', 'SD61', 'SD62', 'SD63',
       'SD64', 'SD65', 'SD66', 'SD67', 'SD68', 'SD69', 'SD70', 'SD71', 'SD72',
       'SD73', 'SD74', 'SD75', 'SD76', 'SD77', 'SD78', 'SD79', 'SD80', 'SD86',
       'SD87', 'SD88', 'SD89', 'SD90', 'SD91', 'SD92', 'SD93', 'SD94', 'SD95',
       'SD96', 'SD97', 'SD98', 'SD99', 'S100', 'S101', 'S102', 'S103', 'S104',
       'S105', 'S106', 'S107'],
      dtype='object', name='site')

In [5]:
pmag_sites = pmag_sites[~((pmag_sites['dir_tilt_correction'] == 100) & (~pmag_sites['site'].isin(leger_sites)))]

In [9]:
pmag_filtered_dir = '../data/pmag'
pmag_contribution.tables['locations'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "locations.txt")
pmag_contribution.tables['sites'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "sites.txt")
pmag_contribution.tables['samples'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "samples.txt")
pmag_contribution.tables['specimens'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "specimens.txt")
pmag_contribution.tables['measurements'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "measurements.txt")

-I- writing locations records to /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt
-I- 1 records written to locations file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/sites.txt
-I- 207 records written to sites file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt
-I- 386 records written to samples file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt
-I- 1435 records written to specimens file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt
-I- 9099 records written to measurements file


'/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt'

## Other issues

Locations (7 Errors)
Location Type (1 Error)
The locations table is missing data in required column "location_type". (Row: 1)
Geologic Setting (1 Error)
The locations table is missing data in required column "geologic_classes". (Row: 1)
Lithologies (1 Error)
The locations table is missing data in required column "lithologies". (Row: 1)
Inferred Age (1 Error)
The locations table is missing data in required column "age" since columns "age_low","age_high" are empty. (Row: 1)
Inferred Age Low (1 Error)
The locations table is missing data in required column "age_low" since column "age" is empty. (Row: 1)
Inferred Age High (1 Error)
The locations table is missing data in required column "age_high" since column "age" is empty. (Row: 1)
Inferred Age Unit (1 Error)
The locations table is missing data in required column "age_unit". (Row: 1)
Sites (10 Errors)
Method Codes (1 Error)
The sites table is missing data in required column "method_codes". (Rows: 11, 18, 65, 296, 319, 324, 401)
Citation DOIs (1 Error)

The sites table is missing data in required column "citations". (Rows: 11, 18, 65, 296, 319, 324, 401)
Geologic Classes (1 Error)
The sites table is missing data in required column "geologic_classes". (Rows: 1-403)
Sample Material Types (1 Error)
The sites table is missing data in required column "geologic_types". (Rows: 1-403)
Lithologies (1 Error)
The sites table is missing data in required column "lithologies". (Rows: 1-403)
Inferred Age (1 Error)
The sites table is missing data in required column "age" since columns "age_low","age_high" are empty. (Rows: 1-403)
Inferred Age Low (1 Error)
The sites table is missing data in required column "age_low" since column "age" is empty. (Rows: 1-403)
Inferred Age High (1 Error)
The sites table is missing data in required column "age_high" since column "age" is empty. (Rows: 1-403)
Inferred Age Unit (1 Error)
The sites table is missing data in required column "age_unit". (Rows: 1-403)
Criteria Names (1 Error)
The sites table column "criteria" value "ACCEPT" is not present in table "criteria" column "criterion". (Rows: 1-10, 12-17, 19-64, 66-295, 297-318, 320-323, 325-400, 402-403)
Specimens (2 Errors)
Method Codes (1 Error)
The specimens table is missing data in required column "method_codes". (Rows: 210, 220, 964)
Citation DOIs (1 Error)
The specimens table is missing data in required column "citations". (Rows: 210, 220, 964)

In [10]:
ipmag.upload_magic(dir_path=pmag_filtered_dir, input_dir_path=pmag_filtered_dir)

-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt file successfully read in
1  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt file successfully read in
386  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt file successfully read in
-I- dropping these columns: location, site from the /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt table
1435  records written to fil

('/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/Zalawt-plain_11.Mar.2025.txt',
 {'status': True,
  'validation': {'errors': [{'table': 'locations',
     'column': 'location_type',
     'message': 'The locations table is missing required column "location_type".',
     'rows': [1]},
    {'table': 'locations',
     'column': 'geologic_classes',
     'message': 'The locations table is missing required column "geologic_classes".',
     'rows': [1]},
    {'table': 'locations',
     'column': 'lithologies',
     'message': 'The locations table is missing required column "lithologies".',
     'rows': [1]},
    {'table': 'locations',
     'column': 'age',
     'message': 'The locations table is missing required column "age" since columns "age_low","age_high" are empty.',
     'rows': [1]},
    {'table': 'locations',
     'column': 'age_low',
     'message': 'The locations table is missing required column "age_low" since column "age" is empty.',
     'rows': [1]},
    {'table': 'l

In [4]:
pmag_sites.tail()

,site,location,samples,specimens,dir_alpha95,dir_comp_name,dir_dec,dir_inc,dir_k,dir_n_samples,dir_n_specimens,dir_n_specimens_lines,dir_n_specimens_planes,dir_polarity,dir_r,dir_tilt_correction,lat,lon,analysts,criteria,software_packages,citations,method_codes,result_quality,vgp_dm,vgp_dp,vgp_lat,vgp_lon,site_name
site name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SD97,SD97,Zalawt plain,SD97-1,SD97-1,NaN,mag,256.9,-46.4,NaN,1.0,1.0,1.0,0.0,t,NaN,100.0,17.1345,54.9548,NaN,ACCEPT,pmagpy-4.2.124: demag_gui.v.3.0,This study,LP-DIR-T:DE-BFL:DA-DIR-TILT:LP-DC4:DA-DIR-GEO,g,231.1,148.3,-19.2,300.8,SD97
SD97,SD97,Zalawt plain,SD97-1,SD97-1,NaN,mag,262.6,-44.2,NaN,1.0,1.0,1.0,0.0,t,NaN,0.0,17.1345,54.9548,NaN,ACCEPT,pmagpy-4.2.124: demag_gui.v.3.0,This study,LP-DIR-T:DE-BFL:DA-DIR-GEO:LP-DC4,g,225.8,141.6,-13.8,301.6,SD97
SD98,SD98,Zalawt plain,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,17.1347,54.9549,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,SD98
SD99,SD99,Zalawt plain,SD99-1:SD99-2:SD99-3,SD99-1:SD99-2:SD99-3,8.1,mag,277.2,-53.1,232.0,3.0,3.0,3.0,0.0,t,2.9914,0.0,17.1348,54.9549,NaN,ACCEPT,pmagpy-4.2.124: demag_gui.v.3.0,This study,LP-DIR-T:DE-BFL:DA-DIR-GEO:LP-DC4:DE-FM,g,11.2,7.8,-3.6,290.7,SD99
SD99,SD99,Zalawt plain,SD99-1:SD99-2:SD99-3,SD99-1:SD99-2:SD99-3,8.1,mag,270.1,-56.6,231.0,3.0,3.0,3.0,0.0,t,2.9914,100.0,17.1348,54.9549,NaN,ACCEPT,pmagpy-4.2.124: demag_gui.v.3.0,This study,LP-DIR-T:DE-BFL:DA-DIR-TILT:LP-DC4:DE-FM:DA-DI...,g,11.7,8.5,-10.2,288.9,SD99
